# Intuitive Algorithm Solver

### Configuration

In [16]:
FILEPATH = r'./../brightspace-locker/Data assignment parcel transport 2 Large.xlsx'

### Initialisations

In [20]:
from integer_linear_problem import ILP
from typing import Dict, List, Set
import pandas as pd
import numpy as np
import copy
import json
from tqdm import tqdm

ilp = ILP(FILEPATH)

32768


### Solver

In [25]:
def powerset(s: Set) -> Set:
    bitmasks = [1 << i for i in range(len(s))] # 1 << i is basically equal to 2^i
    # the n-th digits in the binary representation of bit_selection dictates whether the n-th item is selected (1 = selected, 0 = not selected)
    for bit_selection in range(1, 1 << len(s)): # start with 1 to prevent empty set
        yield {el for el, bitmask in zip(s, bitmasks) if bit_selection & bitmask}

z_min = float('inf')

x = False # TODO: REMOVE
with tqdm(total=(1 << len(ilp.N)) - 1, ncols=100) as pbar:
    for hubs in powerset(ilp.N):
        pbar.update(1)

        non_hubs = ilp.N - hubs

        H = dict()
        for node in ilp.N:
            H[node] = 1 if node in hubs else 0

        E = dict()
        for non_hub in non_hubs:
            E[non_hub] = dict()
            cost_dict = {hub: ilp.c[non_hub][hub] for hub in hubs} # filter on hubs
            min_cost_hub = min(cost_dict, key=cost_dict.get)
            for hub in hubs:
                E[non_hub][hub] = 1 if hub == min_cost_hub else 0

        # current_z = ilp.get_z(H, E)
        # if current_z >= z_min:
        #     continue
        if x:
            continue
        if len(hubs) == 7:# TODO: REMOVE
            x = True
        # z_min = current_z
        # TODO: remove deepcopy and remove copy import
        hubs_min = copy.deepcopy(hubs)
        non_hubs_min = copy.deepcopy(non_hubs)
        H_min = copy.deepcopy(H)
        E_min = copy.deepcopy(E)

print('(Possibly Sub-optimal) Solution:')
print(f'z = {z_min}')
print(f'Hubs = {hubs_min}')
print(f'Non-hubs = {non_hubs_min}')
print(f'H = {json.dumps(H_min, indent=4)}')
print(f'E = {json.dumps(E_min, indent=4)}')
print(ilp)

# TODO: add pyvis, tqdm or whatever to README.md
# TODO: html files in graphs directory with name ending in excel file name  -> in graph class convert file name to absolute with os library and set parameter filepath=None as default

100%|██████████████████████████████████████████████████████| 32767/32767 [00:03<00:00, 10655.95it/s]

(Possibly Sub-optimal) Solution:
z = inf
Hubs = {1, 2, 3, 4, 5, 6, 7}
Non-hubs = {8, 9, 10, 11, 12, 13, 14, 15}
H = {
    "1": 1,
    "2": 1,
    "3": 1,
    "4": 1,
    "5": 1,
    "6": 1,
    "7": 1,
    "8": 0,
    "9": 0,
    "10": 0,
    "11": 0,
    "12": 0,
    "13": 0,
    "14": 0,
    "15": 0
}
E = {
    "8": {
        "1": 0,
        "2": 0,
        "3": 0,
        "4": 0,
        "5": 0,
        "6": 0,
        "7": 1
    },
    "9": {
        "1": 0,
        "2": 0,
        "3": 0,
        "4": 0,
        "5": 1,
        "6": 0,
        "7": 0
    },
    "10": {
        "1": 0,
        "2": 0,
        "3": 0,
        "4": 0,
        "5": 0,
        "6": 0,
        "7": 1
    },
    "11": {
        "1": 0,
        "2": 0,
        "3": 0,
        "4": 0,
        "5": 0,
        "6": 0,
        "7": 1
    },
    "12": {
        "1": 0,
        "2": 0,
        "3": 0,
        "4": 0,
        "5": 0,
        "6": 0,
        "7": 1
    },
    "13": {
        "1": 0,
        "2": 